# Coordinate da Indirizzo

In [2]:
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent = "myapp")
location = locator.geocode("Via Novegro 20090 Segrate")
start_lat=location.latitude 
start_lng=location.longitude
print(start_lat,start_lng)


45.4688535 9.2776371


In [12]:
import geopy.geocoders
locator = Nominatim(user_agent = "myapp")
location = locator.geocode("Via Como 9 Barzago")
end_lat=location.latitude 
end_lng=location.longitude
print(end_lat,end_lng)

45.7578372 9.3097403


# Calcolo percorso più veloce

In [14]:
#Find Distance
import osmnx as ox
import networkx as nx
import numpy as np
from IPython.display import IFrame
import folium
%matplotlib inline

np.random.seed(0)
ox.__version__

ox.settings.log_console=True
ox.settings.cache_folder='C:\\Users\\lucaf\\OneDrive\\Desktop\\Esercizi\\Cache'
place = "Milan,Lombardy,Italy"
mode="drive"
G = ox.graph_from_place(place, network_type=mode)
Gp = ox.project_graph(G)

KeyboardInterrupt: 

In [85]:
orig_point = (start_lat, start_lng) # UTM coordinates
dest_point = (end_lat, end_lng)

orig_node, dest_node = ox.nearest_nodes(
                            Gp,
                            X=[orig_point[0], dest_point[0]],
                            Y=[orig_point[1], dest_point[1]]
                                       )

In [86]:
orig_edge, dest_edge = ox.nearest_edges(
                            Gp,
                            X=[orig_point[0], dest_point[0]],
                            Y=[orig_point[1], dest_point[1]]
                                       )

In [87]:
print(orig_point,dest_point)
print(orig_node,dest_node)

(45.4688535, 9.2776371) (45.7578372, 9.3097403)
583043033 583043033


In [80]:
route = ox.shortest_path(Gp, orig_node, dest_node, weight="length")

In [13]:
shortest_route_map = ox.plot_route_folium(G, route,tiles='openstreetmap')
shortest_route_map

NameError: name 'ox' is not defined

In [ ]:

# see mean speed/time values by road type
edges = ox.graph_to_gdfs(G, nodes=False)
edges["highway"] = edges["highway"].astype(str)
edges.groupby("highway")[["length", "speed_kph", "travel_time"]].mean().round(1)

# same thing again, but this time pass in a few default speed values (km/hour)
# to fill in edges with missing `maxspeed` from OSM
hwy_speeds = {"residential": 35, "secondary": 50, "tertiary": 60}
G = ox.add_edge_speeds(G, hwy_speeds)
G = ox.add_edge_travel_times(G)

# calculate two routes by minimizing travel distance vs travel time
orig = list(G)[1]
dest = list(G)[120]
route1 = ox.shortest_path(G, orig, dest, weight="length")
route2 = ox.shortest_path(G, orig, dest, weight="travel_time")

# compare the two routes
route1_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "length")))
route2_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, "length")))
route1_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "travel_time")))
route2_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, "travel_time")))
print("Route 1 is", route1_length, "meters and takes", route1_time, "seconds.")
print("Route 2 is", route2_length, "meters and takes", route2_time, "seconds.")


In [2]:
import pandas as pd
import osmnx as ox
import networkx as nx
import numpy as np
from IPython.display import IFrame
import folium
import pandas as pd
import osmnx as ox

ox.settings.cache_folder='C:\\Users\\lucaf\\OneDrive\\Desktop\\Esercizi\\Cache'

places=["Milan,Lombardy,Italy","Segrate,Lombardy,Italy"]
tags = {"public_transport":"platform",
        "bus":"yes",
        "node":True}
all_pois = ox.geometries_from_place(places,tags)
point=pd.DataFrame(all_pois)
point=point[["name","public_transport","geometry"]].reset_index()
point=point[point.element_type=="node"]
point["lat"]=point.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[1])
point["lon"]=point.geometry.apply(lambda x:str(x).split("(")[1].replace(")","").split(" ")[0])
point=point.drop_duplicates(subset=['lat','lon'])

In [4]:
autobus=point
autobus["icona"]={'url':'https://upload.wikimedia.org/wikipedia/commons/a/a7/Map_marker_icon_–_Nicolas_Mollet_–_Subway_–_Transportation_–_Default.png','width':32,'height':37,'anchorY':37}

In [6]:
autobus.to_csv("locations_tomap.csv")